In [98]:
import csv
import requests
import os
import numpy as np
import pandas as pd 
from urllib.request import urlopen
from bs4 import BeautifulSoup
import geocoder 
from sklearn.cluster import KMeans
import folium 
import matplotlib.cm as cm
import matplotlib.colors as colors

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

### Part 1: Text Mining the Information

Used BS4 to mine the information. First found all elements that had <td> and subsequently made the rows for the dataframe

In [2]:
### Header Detail
headers = { 'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

with requests.Session() as s :
    r = s.get(link, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')

    data = soup.find_all("td")


In [3]:
### Seeing content of Text Mined
for line in enumerate(data) : 
    print(line[1].text.rstrip())
    print(line[0])
    
    if line[0] == 5 : 
        break 

M1A
0
Not assigned
1
Not assigned
2
M2A
3
Not assigned
4
Not assigned
5


In [4]:
# Create Panda DF 
neigh = pd.DataFrame(columns=['Postcode', 'Borough', 'Neighborhood'] )
neigh.head()

,Postcode,Borough,Neighborhood


In [5]:
for i in range(0,len(data),3) : 
    postcode = data[i].text.rstrip()
    borough = data[i+1].text.rstrip()
    neighborhood = data[i+2].text.rstrip()
    
    ### Check if this is the last row 
    if postcode == "M9Z" : 
        break 
    
    ### Check if borough is assigned
    if borough == "Not assigned" :
        continue 
        
    
    ### Check if postcode exists already
    if postcode in neigh.values :
        neigh.at[len(neigh)-1, "Neighborhood"] = neigh.at[len(neigh)-1, "Neighborhood"] + ", " + neighborhood
        
    else :  
        
        if neighborhood == "Not assigned" :
            neighborhood = borough 
            
        neigh = neigh.append({'Postcode': postcode,
                              'Borough': borough,
                              'Neighborhood': neighborhood}, ignore_index=True)



In [6]:
neigh.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [7]:
neigh.shape

(103, 3)

### Part 2: Adding Latitude and Longitude Coordinates

Unable to retrieve the latitude and longitude coordinates for each Postcode. Loaded in the data from the given csv file.

In [ ]:
full_lat = []
full_long = []

for row in range( len(neigh) ) : 
    postal_code = neigh.iloc[row]["Postcode"]
    print(postal_code)
    
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    
    full_lat.append(latitude)
    full_long.append(longitude)
    

M3A


Status code Unknown from https://maps.googleapis.com/maps/api/geocode/json: ERROR - HTTPSConnectionPool(host='maps.googleapis.com', port=443): Read timed out. (read timeout=5.0)
Status code Unknown from https://maps.googleapis.com/maps/api/geocode/json: ERROR - HTTPSConnectionPool(host='maps.googleapis.com', port=443): Read timed out. (read timeout=5.0)


In [9]:
test = pd.read_csv("Geospatial_Coordinates.csv")

In [18]:
test

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [20]:
lat_val = test.Latitude.values
long_val = test.Longitude.values

neigh["Latitude"] = lat_val 
neigh["Longitude"] = long_val 

In [22]:
neigh.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,Harbourfront,43.763573,-79.188711
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.770992,-79.216917
4,M7A,Downtown Toronto,Queen's Park,43.773136,-79.239476


### Part 3: K-Means Analysis

One of the district name was named Neighborhood so had to rename Neighborhood to Neighborhood_Name to prevent conflict. Similiar to previous lab, cluster attempts to find similiar/common venue districts. Cluster 3 had the largest amount of points and seems to be finding neighborhoods that contains restaurants, shopping districts, relaxing activities, etc. Cluster 1 attemps to find parks. Cluster 0 seems to primary focuse on restaurants/food districts. Cluster 2 classification is a bit unclear as it only has one classification. Cluster 4 seems to be focusing on baseball fields. In general, due to large number of classification in Cluster 3, this model might be underfitting and we would either need more neighborhood points or increase/decrease the number of clusters present. 

**Secret Client ID/Key/Version were removed when pushed to Github

In [32]:
toronto_df = pd.DataFrame(columns=['Postcode', 'Borough', 'Neighborhood', "Latitude", "Longitude"] )

for row in range( len(neigh) ) : 
    if "Toronto" in neigh.iloc[row]["Borough"] : 
        toronto_df = toronto_df.append(neigh.iloc[row], ignore_index=True)

In [33]:
toronto_df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.763573,-79.188711
1,M7A,Downtown Toronto,Queen's Park,43.773136,-79.239476
2,M5B,Downtown Toronto,"Ryerson, Garden District",43.692657,-79.264848
3,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389
4,M4E,East Toronto,The Beaches,43.786947,-79.385975


In [43]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = ''
LIMIT = 30
radius = 500

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [44]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )


Harbourfront
Queen's Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

In [62]:
print(toronto_venues.shape)
toronto_venues.head()

(476, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
1,Harbourfront,43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa
2,Harbourfront,43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
3,Harbourfront,43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
4,Harbourfront,43.763573,-79.188711,Wood Floor Polishing Inc,43.766500,-79.185207,Moving Target


In [59]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 158 uniques categories.


In [66]:
toronto_venues.rename( columns={"Neighborhood": "Neighborhood_Name"}, inplace=True)

In [67]:
# One hot encode 
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# Add neighborhood column and shift to front
toronto_onehot['Neighborhood_Name'] = toronto_venues['Neighborhood_Name'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood_Name,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Baby Store,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Store,Belgian Restaurant,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Cuban Restaurant,Curling Ice,Deli / Bodega,Dessert Shop,Diner,Discount Store,Dog Run,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Latin American Restaurant,Light Rail Station,Liquor Store,Market,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,Nightclub,Noodle House,Organic Grocery,Park,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Salad Place,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Snack Place,Spa,Sporting Goods Shop,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [68]:
toronto_onehot.shape

(476, 159)

In [71]:
toronto_grouped = toronto_onehot.groupby('Neighborhood_Name').mean().reset_index()
toronto_grouped.head()

,Neighborhood_Name,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Baby Store,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Store,Belgian Restaurant,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Cuban Restaurant,Curling Ice,Deli / Bodega,Dessert Shop,Diner,Discount Store,Dog Run,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Latin American Restaurant,Light Rail Station,Liquor Store,Market,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,Nightclub,Noodle House,Organic Grocery,Park,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Salad Place,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Snack Place,Spa,Sporting Goods Shop,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.333333,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0000,0.00,0.0000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.333333,0.000000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.000000,0.0,0.000000,0.0,0.0000,0.0,0.333333,0.0000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000
1,Berczy Park,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0000,0.00,0.0000,1.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.000000,0.0,0.000000,0.0,0.0000,0.0,0.000000,0.0000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.

In [73]:
toronto_grouped.shape

(39, 159)

In [75]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood_Name']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood_Name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                       venue  freq
0                    Airport  0.33
1                       Park  0.33
2                Snack Place  0.33
3  Latin American Restaurant  0.00
4         Light Rail Station  0.00


----Berczy Park----
                venue  freq
0           Cafeteria   1.0
1             Airport   0.0
2   Mobile Phone Shop   0.0
3  Light Rail Station   0.0
4        Liquor Store   0.0


----Brockton, Exhibition Place, Parkdale Village----
                venue  freq
0                Café  0.13
1         Coffee Shop  0.10
2  Italian Restaurant  0.07
3              Bakery  0.07
4             Brewery  0.03


----Business Reply Mail Processing Centre 969 Eastern----
                       venue  freq
0          Mobile Phone Shop  0.25
1                Pizza Place  0.25
2             Sandwich Place  0.25
3                   Bus Line  0.25
4  Middle Eastern Restaurant  0.00


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and

In [76]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [92]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood_Name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood_Name'] = toronto_grouped['Neighborhood_Name']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood_Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Airport,Park,Snack Place,Cuban Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Diner,Dessert Shop,Deli / Bodega
1,Berczy Park,Cafeteria,Yoga Studio,Curling Ice,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Diner,Dessert Shop,Deli / Bodega
2,"Brockton, Exhibition Place, Parkdale Village",Café,Coffee Shop,Bakery,Italian Restaurant,Brewery,Bar,Comfort Food Restaurant,Sandwich Place,Bookstore,Ice Cream Shop
3,Business Reply Mail Processing Centre 969 Eastern,Bus Line,Pizza Place,Mobile Phone Shop,Sandwich Place,Coworking Space,Discount Store,Diner,Dessert Shop,Deli / Bodega,Curling Ice
4,"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",Yoga Studio,Spa,Fast Food Restaurant,Farmers Market,Light Rail Station,Comic Shop,Garden Center,Park,Pizza Place,Butcher


In [93]:
kclusters = 5
toronto_clustering = toronto_grouped.drop('Neighborhood_Name', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 3, 0, 3, 0, 0, 3, 1, 0])

In [94]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood_Name'), on='Neighborhood')

toronto_merged.head() 

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.763573,-79.188711,3,Spa,Moving Target,Electronics Store,Mexican Restaurant,Breakfast Spot,Medical Center,Intersection,Rental Car Location,Yoga Studio,Deli / Bodega
1,M7A,Downtown Toronto,Queen's Park,43.773136,-79.239476,3,Gas Station,Fried Chicken Joint,Bank,Bakery,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Caribbean Restaurant,Concert Hall,Eastern European Restaurant
2,M5B,Downtown Toronto,"Ryerson, Garden District",43.692657,-79.264848,3,College Stadium,Café,Skating Rink,General Entertainment,Comfort Food Restaurant,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store
3,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389,3,Fast Food Restaurant,Chinese Restaurant,Coffee Shop,Sandwich Place,Breakfast Spot,Pharmacy,Thrift / Vintage Store,Pizza Place,Grocery Store,Supermarket
4,M4E,East Toronto,The Beaches,43.786947,-79.385975,3,Chinese Restaurant,Bank,Café,Japanese Restaurant,Yoga Studio,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store


In [100]:
# create map
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [101]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown Toronto,0,Coffee Shop,Discount Store,Pharmacy,Pizza Place,Coworking Space,Dog Run,Diner,Dessert Shop,Deli / Bodega,Curling Ice
28,West Toronto,0,Bus Line,Pizza Place,Grocery Store,Convenience Store,Yoga Studio,Dumpling Restaurant,Dog Run,Discount Store,Diner,Dessert Shop
35,Downtown Toronto,0,Empanada Restaurant,Pizza Place,Cuban Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Diner,Dessert Shop,Deli / Bodega,Curling Ice
37,Downtown Toronto,0,Pizza Place,Coffee Shop,Chinese Restaurant,Intersection,Sandwich Place,Curling Ice,Dog Run,Discount Store,Diner,Dessert Shop
38,East Toronto,0,Bus Line,Pizza Place,Mobile Phone Shop,Sandwich Place,Coworking Space,Discount Store,Diner,Dessert Shop,Deli / Bodega,Curling Ice


In [102]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,1,Park,Food & Drink Shop,Electronics Store,Dumpling Restaurant,Dog Run,Discount Store,Diner,Dessert Shop,Deli / Bodega,Curling Ice
8,Downtown Toronto,1,Airport,Park,Snack Place,Cuban Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Diner,Dessert Shop,Deli / Bodega
24,Central Toronto,1,Park,Market,Women's Store,Garden,Dumpling Restaurant,Discount Store,Diner,Dessert Shop,Garden Center,Deli / Bodega


In [103]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Downtown Toronto,2,Cafeteria,Yoga Studio,Curling Ice,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Diner,Dessert Shop,Deli / Bodega


In [104]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,3,Spa,Moving Target,Electronics Store,Mexican Restaurant,Breakfast Spot,Medical Center,Intersection,Rental Car Location,Yoga Studio,Deli / Bodega
1,Downtown Toronto,3,Gas Station,Fried Chicken Joint,Bank,Bakery,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Caribbean Restaurant,Concert Hall,Eastern European Restaurant
2,Downtown Toronto,3,College Stadium,Café,Skating Rink,General Entertainment,Comfort Food Restaurant,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store
3,Downtown Toronto,3,Fast Food Restaurant,Chinese Restaurant,Coffee Shop,Sandwich Place,Breakfast Spot,Pharmacy,Thrift / Vintage Store,Pizza Place,Grocery Store,Supermarket
4,East Toronto,3,Chinese Restaurant,Bank,Café,Japanese Restaurant,Yoga Studio,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store
9,West Toronto,3,Grocery Store,Shopping Mall,Park,Hotel,Bank,Curling Ice,Dumpling Restaurant,Dog Run,Discount Store,Diner
10,Downtown Toronto,3,Skating Rink,Beer Store,Park,Pharmacy,Cosmetics Shop,Asian Restaurant,Curling Ice,Yoga Studio,Deli / Bodega,Dumpling Restaurant
11,West Toronto,3,Neighborhood,Coffee Shop,Trail,Health Food Store,Pub,Cuban Restaurant,Dog Run,Discount Store,Diner,Dessert Shop
12,East Toronto,3,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Yoga Studio,Coffee Shop,Brewery,Restaurant,Bubble Tea Shop,Pub,Pizza Place
13,Downtown Toronto,3,Park,Ice Cream Shop,Fast Food Restaurant,Brewery,Movie Theater,Pub,Burrito Place,Steakhouse,Pizza Place,Sushi Restaurant


In [106]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Downtown Toronto,4,Baseball Field,Yoga Studio,Empanada Restaurant,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Diner,Dessert Shop,Deli / Bodega
36,Downtown Toronto,4,Baseball Field,Furniture / Home Store,Yoga Studio,Curling Ice,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Diner,Dessert Shop
